In [1]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # 특정 GPU에 1GB 메모리만 할당하도록 제한
    try:
        tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
        tf.config.experimental.set_virtual_device_configuration(
            gpus[1],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1048)])
    except RuntimeError as e:
    # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
        print(e)

In [2]:
import os
import pandas as pd
from glob import glob
import random
import shutil
import matplotlib.pyplot as plt
import numpy as np


In [3]:
path_train = './data/public/train/'

path_cate_csv = './data/public/category.csv'

df_cate = pd.read_csv(path_cate_csv)

df_cate.head()
                          

,landmark_id,landmark_name
0,0,금천구청
1,1,두리랜드
2,2,이대원 장군묘
3,3,성공회대학교
4,4,송상현공 동상


In [4]:
df_dict = dict(df_cate.values[:,::-1]) # 데이터 프레임 안의 내용 dict로 받음

In [5]:
train_list = glob(os.path.join(path_train, '*', '*', '*.JPG'))
len(train_list)

0

In [6]:
def load_class_idx_from_path(img_path, category_dict): 
    # DongminChoi 님 코드 공유에서 함수 잘 만들어 놓으셔서 가져왔습니다.
    
    '''
    Load class index from data path based on class name
    '''
    filename = os.path.basename(img_path)
    class_name = ' '.join(filename.split('_')[:-1])
    
    # 이케아_광명, 이케아_고양 폴더명에 _가 포함되어 있어 따로 다시 class_name을 처리해줍니다.
    
    print(filename)
    print(class_name)
    if class_name =='이케아 광명':
        class_name ='이케아_광명'
    if class_name =='이케아 고양':
        class_name = '이케아_고양'
    return category_dict[class_name]


path_train_new = os.path.join('./data/public/train_new/')


def make_folder(folder_name): # 폴더 생성 함수

    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
        
make_folder(path_train_new)


# 미리 class_id 이름으로 이동시킬 폴더를 생성합니다.

for i in range(len(df_cate)):
    path_train_class = os.path.join(path_train_new, str(i))
    
    make_folder(path_train_class)


# 기존 train에서 class_id로 만들어진 새로운 train_new 폴더로 이동(복사 아님) 시킵니다.
    
for i, image in enumerate(train_list):
    

    idx = image.rfind('/')
    image_name = image[idx:]

    class_id = load_class_idx_from_path(image, df_dict)
    path_shutil = os.path.join(path_train_new, '%d'%class_id  + image_name)

    
    shutil.move(image, path_shutil)


In [ ]:
##########################################################################################################

In [9]:
train_list = glob(os.path.join(path_train_new, '*', '*.JPG'))
len(train_list)
dir_list = sorted(glob(os.path.join(path_train_new, '*')))
print(len(dir_list))

1049


In [13]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

train_dir = './data/public/train_new'
# validation_dir = './data/cat_dog_full/validation'

# 데이터 준비
# ImageDataGenerator를 생성
# 이미지 데이터의 값을 1/255로 scailing
train_datagen = ImageDataGenerator(rescale=1/255)

batch_size=20
classes=[str(x) for x in range(len(dir_list))]
# print(classes)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    classes=classes,
    target_size=(150,150),
    batch_size=batch_size, 
    class_mode='categorical')

nr_of_classes = len(train_generator.class_indices)
print(nr_of_classes)


['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '15

In [ ]:
# 모델 생성
with tf.device('/device:GPU:1'):
    
    model = Sequential()
    
    # feature 뽑아내자!
    model.add(Conv2D(filters=32,
                     kernel_size=(3,3),
                     activation='relu',
                     input_shape=(150,150,3))) 
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Conv2D(filters=64,
                     kernel_size=(3,3),
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Conv2D(filters=128,
                     kernel_size=(3,3),
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    # 학습 진행    
    model.add(Flatten())
    
    model.add(Dropout(0.5))
    
    model.add(Dense(units=65,
                    activation='relu'))
    
    model.add(Dense(units=nr_of_classes,
                    activation='softmax'))
    
    print(model.summary())
    
    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    history = model.fit(train_generator,
                        steps_per_epoch=len(train_list)/batch_size,
                        epochs=30)
                        
model.save('./landmark_cnn_model.h5')    # 학습된 결과 저장

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 36992)             0